## Génération d'un tableau de coordonnées fictives

In [2]:
import numpy as np

atomes_nb = 4
atomes_nb_incl_fictif = atomes_nb + 1

# Génération d'un vecteur de position des atomes d'une molécule. Pour atome, 
# le vecteur contient 3 coordonnées x, y, z, chacune dans l'intervalle [-1,1[. 

positions = np.random.random_sample(atomes_nb*3) * 2 - 1

positions

array([-0.62100738, -0.80278759,  0.59135765,  0.43723662, -0.50545906,
       -0.90807326, -0.16368264,  0.4612981 ,  0.66805421, -0.76304705,
        0.75725923,  0.01481533])

## Conversion des coordonnées en distances

### Ajout d'un atome fictif

In [3]:
def ajout_atome_fictif(positions):
    """ Ajoute un atome fictif aux positions initiales. Pour le moment, le place simplement
    aux coordonnées (0,0,0). Dans le futur, elle le placera à une position n'appartenant pas avec
    certitude au plan formé par les trois premiers atomes. """
    return np.insert(positions, 0, [0,0,0])
    
positions = ajout_atome_fictif(positions)
positions

array([ 0.        ,  0.        ,  0.        , -0.62100738, -0.80278759,
        0.59135765,  0.43723662, -0.50545906, -0.90807326, -0.16368264,
        0.4612981 ,  0.66805421, -0.76304705,  0.75725923,  0.01481533])

### Génération du vecteur distances

In [4]:
import math

def pos_to_index(i, j):
    """ Prend les coordonnées d'une entrée dans la matrice de distances et renvoie l'index de la distance
    correspondante dans le vecteur de distances. """
    return i*4+j-1

def get_atome_coord(positions, index):
    """ Renvoie les coordonnées de l'atome ayant l'index donné """
    return ((positions[3*index], positions[3*index+1], positions[3*index+2]))
    
def calcul_dist(x1, y1, z1, x2, y2, z2):
    """ Calcule et retourne la distance entre deux points selon leurs coordonnées en 3 dimensions"""

    return math.sqrt(pow(x1 - x2, 2) + pow(y1 - y2, 2) + pow(z1 - z2, 2))
    
def get_val_dist_matrice(i, j, positions):
    """ Renvoie la valeur de la matrice de distances aux coordonnées (i, j). La matrice complète n'est
    jamais calculée. """
    coord_i = get_atome_coord(positions, i)  # Coordonnées de l'atome i
    coord_j = get_atome_coord(positions, j)  # Coordonnées de l'atome j

    return calcul_dist(coord_i[0], coord_i[1], coord_i[2], coord_j[0], coord_j[1], coord_j[2])
    
def vect_distances(positions, nb_atomes):
    """ Renvoie le vecteur de distances à partir des coordonnées des atomes et du nombre d'atomes dans
    la molécule. La matrice de distances complète n'est pas générée, seules les valeurs nécessaires
    sont calculées à la demande """
    vect_distances = np.zeros(shape=((nb_atomes-1)*4))
    for j in range(nb_atomes - 1):
        k=0  # Permet d'accéder simplement à l'indice dans le tableau de sortie
        for i in range(j+1, j+5):
            if i < nb_atomes:
                vect_distances[j*4+k] = get_val_dist_matrice(i, j, positions)
            k += 1
            
    return vect_distances
            

vect_distances(positions, atomes_nb_incl_fictif)

array([1.17465823, 1.12750245, 0.82818137, 1.07512875, 1.85918737,
       1.34645495, 1.66922808, 0.        , 1.94419679, 1.97151254,
       0.        , 0.        , 0.93463989, 0.        , 0.        ,
       0.        ])

### Test de la génération du vecteur distances

In [5]:
# On génère un tableau de positions d'atomes
pos = [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5]

# On ajoute l'atome fictif
pos = ajout_atome_fictif(pos)

# Valeurs attendues du vecteur distances
# [d10, d20, d30, d40,
#  d21, d31, d41, d51,
#  d32, d42, d52, 0,
#  d43, d53, 0,   0,
#  d54, 0,   0,   0]
#
# d10 = 1.73...
# d20 = 3,46...
# d30 = 5,19...
# d40 = 6,92...
# d21 = 1,73...
# d31 = 3,46...
# d41 = 5,19...
# d51 = 6,92...
# d32 = 1,73...
# d42 = 3,46...
# d52 = 5,19...
# d43 = 1.73...
# d53 = 3,46...
# d54 = 1,73...

vect_distances(pos, 6)

# Les valeurs de retour sont bien celles attendues

array([1.73205081, 3.46410162, 5.19615242, 6.92820323, 1.73205081,
       3.46410162, 5.19615242, 6.92820323, 1.73205081, 3.46410162,
       5.19615242, 0.        , 1.73205081, 3.46410162, 0.        ,
       0.        , 1.73205081, 0.        , 0.        , 0.        ])